In [2]:
import base64
import re
import math  
from Crypto.PublicKey import RSA

# This is the RSA public key using OpenSSL

key_encoded='''-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA3Xo7Coo024M3KeQaDAsn
yKWf9f0356HV1vDI5QljXVA1txvJ2uh/BpcCqz+Q72l49p88k09q7HnnI4E87LDy
/ouJSabERStUyGM1oDtwj3Q4AtnGZpkkcwL0Dd14ytOwlDrR1JNNZy8a6rC6VH82
3GHChEi2g6BerHWClj2Fjd0tuGP6pDM/3bBaFffG2x2Kk0RxYa5ieYeq9VDG3JWb
iwieS9/6xI+NuJ7OUi06Z6pR+KP9cdK3f2FandOxmGX5we7Zjr/HZehPI1tCRRmF
3fdebIVuK3divsmxsDUCiyvXtNRtlWL0nxwpvP8/sKx/i8FEN+rB3y2c8rPj138u
5QIDAQAB
-----END PUBLIC KEY-----'''

pubkey = RSA.importKey(key_encoded)

# N = 90581                 # For testing purposes, to see if Wiener's Attack is implemented correctly
# e = 17993 

N = pubkey.n                # Export N from the RSA public key
print("N = " + str(N))
e = pubkey.e                # Export e from the RSA public key
print("\ne = " + str(e))

bitsOfN = bin(N)
print("\nRSA length = " + str(len(bitsOfN)-2))         # Length - 2 to get rid of the "0b" binary initials


'''Everything below is code I have written to implement Wiener's attack on exercise 11'''


# The fast() function from exercise 8(iv) to compute remainder of ( a^g modulo N )

def fast(a,g,N):
    g = bin(g)         # Turn to binary
    g = g[2:]          # Remove the 0b at the beginning

    d = 1
    x = a
    for i in range (len(g)-1,-1,-1):
        if g[i]==str(1):
            d = (d * x) % N
        x = (x ** 2) % N
    return d


# Here i calculate the continued fraction of e/N

def continued_fraction(e, N):
    
    integers=[]
    while N>0:
        a = e // N
        integers.append(a)
        temp = e
        e = N
        N = temp - a * N
    return integers


# Here I compute the convergents based on the continued fraction list given.

def convergents(frac_list):

    convs = [];
    for i in range (0,len(frac_list)):
        convs.append(formula(frac_list[0:i+1]))
    return convs


# num = numerator, denom = denominator

def formula (frac_list):
    
    num = frac_list[-1]
    den = 1
    for i in range(-2,-len(frac_list)-1,-1):
        num, den = frac_list[i]*num+den, num
    return (num,den)


# Here i build the Wiener's attack algorithm.
# To check if d is actually the key, I need to compute x^2 - (N - phi + 1)x + N = 0
# If it has 2 integer roots, then we have successfully found d.

def find_d (convs,e,n):
    
    for (k,d) in convs:
        if k!=0 and (e*d-1)%k == 0:         # If phi ( (e*d - 1) / N ) is an integer
            phi = (e*d-1)//k
            b = n - phi + 1
            delta = b*b - 4*1*n             # discriminant = b^2 - 4*a*g
            if(delta>=0):                   # If discriminant is an integer
                t = math.sqrt(delta)
                if t*t == delta and (b+t)%2==0:
                    return d                # We successfully found d

                
# Find continued fraction of e/N, find its convergents, and finally find the d
                
confrac = continued_fraction(e,N)
convs = convergents(confrac)
d = find_d(convs,e,N)

print("\n...Calculating d...")

if (d!=None):
    print("\nD = " + str(d))
else:
    print("\nWiener's attack FAILED. Couldn't calculate d.")

N = 27958939567391778921189107684959846921237291970695610753872060492639871398714497243139337468556282256593113648569670337961342208568511903691177976892010570422172879646290355948393344560582811954238229422002342558494893462674873406853251524812726955405882738280604306038811109213379162197383538140267905834699689284696671847006303067923779160924662546102771569188249977323100097766859805858351298056861892053453904228371475617041342423188940362215676179680801541078666772428861231001258658408914188390741703705879437116975950213625822691090557014457932298907047419561590098042318776936554854728901351383666037746511589

e = 65537

RSA length = 2048

...Calculating d...

Wiener's attack FAILED. Couldn't calculate d.
